In [1]:
import tensorflow as tf
import numpy as np

2024-12-12 15:19:14.398091: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
BATCH_SIZE = 32
IMAGE_SIZE = (224, 224)
NUM_MODELS = 5
EPOCHS = 10
DATA_SOURCE_TRAIN = 'data/malhari/train'
DATA_SOURCE_TEST = 'data/malhari/test'

In [3]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_SOURCE_TRAIN,
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_SOURCE_TEST,
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False,
)

class_names = train_ds.class_names

Found 2630 files belonging to 3 classes.
Found 297 files belonging to 3 classes.


In [4]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

In [5]:
# Subsetting images by (image, label) pairs.
train_images = []
train_labels = []
for x, y in train_ds.unbatch():
    train_images.append(x)
    train_labels.append(y.numpy())

train_images = tf.stack(train_images)
train_labels = np.array(train_labels)

2024-12-12 15:19:35.263609: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [6]:
def create_subset_dataset(images, labels, subset_size):
    # Sample indices with replacement
    indices = np.random.choice(len(labels), size=subset_size, replace=True)
    sampled_images = tf.gather(images, indices)
    sampled_labels = tf.gather(labels, indices)

    # Build a dataset
    ds = tf.data.Dataset.from_tensor_slices((sampled_images, sampled_labels))

    # Data augmentation (optional)
    data_augmentation = tf.keras.Sequential([
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.1),
    ])

    # Preprocessing function
    def preprocess(image, label):
        image = tf.image.resize(image, IMAGE_SIZE)
        image = tf.keras.applications.vgg16.preprocess_input(image)
        image = data_augmentation(image)
        return image, label

    ds = ds.map(preprocess, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return ds

In [7]:
def create_vgg16_model(num_classes=len(class_names)):
    base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=IMAGE_SIZE + (3,))
    base_model.trainable = False  # Freeze the base model layers

    # Add custom top layers
    x = tf.keras.layers.Flatten()(base_model.output)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.models.Model(inputs=base_model.input, outputs=outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [8]:
def test_preprocess(image, label):
    image = tf.image.resize(image, IMAGE_SIZE)
    image = tf.keras.applications.vgg16.preprocess_input(image)
    return image, label

test_ds_processed = val_ds.map(test_preprocess, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
subset_size = len(train_labels)

In [10]:
# Train multiple models on different subsets of the training data
ensemble_models = []
for i in range(NUM_MODELS):
    print(f"Training model {i+1}/{NUM_MODELS}...")
    callbacks = [
        tf.keras.callbacks.ModelCheckpoint(
            f'checkpoints/malhari_vgg16_encemble_{i}.keras',
            monitor='val_loss',
            save_best_only=True,
            verbose=1
        ),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
            verbose=1
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=5,
            min_lr=1e-6,
            verbose=1
        )
    ]
    model = create_vgg16_model(num_classes=len(class_names))
    subset_ds = create_subset_dataset(train_images, train_labels, subset_size)
    history = model.fit(subset_ds, validation_data=test_ds_processed, epochs=EPOCHS, callbacks=callbacks)
    ensemble_models.append(model)
    print(history)

Training model 1/5...
Epoch 1/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.4820 - loss: 22.1672
Epoch 1: val_loss improved from inf to 0.76257, saving model to checkpoints/malhari_vgg16_encemble_0.keras
83/83 ━━━━━━━━━━━━━━━━━━━━ 382s 5s/step - accuracy: 0.4826 - loss: 22.0065 - val_accuracy: 0.6364 - val_loss: 0.7626 - learning_rate: 0.0010
Epoch 2/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.6696 - loss: 0.7201
Epoch 2: val_loss improved from 0.76257 to 0.51145, saving model to checkpoints/malhari_vgg16_encemble_0.keras
83/83 ━━━━━━━━━━━━━━━━━━━━ 580s 7s/step - accuracy: 0.6698 - loss: 0.7197 - val_accuracy: 0.7778 - val_loss: 0.5115 - learning_rate: 0.0010
Epoch 3/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.7663 - loss: 0.5388
Epoch 3: val_loss improved from 0.51145 to 0.42714, saving model to checkpoints/malhari_vgg16_encemble_0.keras
83/83 ━━━━━━━━━━━━━━━━━━━━ 451s 5s/step - accuracy: 0.7663 - loss: 0.5386 - val_accuracy: 0.8316 - val_loss: 0.4271 - 

In [11]:
def ensemble_predict(models, dataset):
    all_preds = []
    for m in models:
        preds = m.predict(dataset, verbose=0)
        all_preds.append(preds)
    avg_preds = np.mean(np.array(all_preds), axis=0)
    return avg_preds

In [12]:
# Get ground truth from test set
y_true = []
test_images_list = []
for img, lbl in val_ds.unbatch():
    test_images_list.append(img)
    y_true.append(lbl.numpy())

test_images_list = tf.stack(test_images_list)
y_true = np.array(y_true)

# Preprocess test images for prediction
test_images_prep = tf.keras.applications.vgg16.preprocess_input(
    tf.image.resize(test_images_list, IMAGE_SIZE)
)

test_predict_ds = tf.data.Dataset.from_tensor_slices(test_images_prep).batch(BATCH_SIZE)

ensemble_predictions = ensemble_predict(ensemble_models, test_predict_ds)
y_pred = np.argmax(ensemble_predictions, axis=1)

accuracy = np.mean(y_pred == y_true)
print("Ensemble test accuracy:", accuracy)

2024-12-12 21:03:23.697113: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Ensemble test accuracy: 0.9663299663299664
